In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [36]:
class ConvNet(nn.Module):
    def __init__(self, in_channels= 1, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d( kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        
        return x

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [38]:
in_channel = 1
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [39]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)



In [40]:
model = ConvNet().to(device)

In [41]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [42]:
for epoch in range(num_epochs):
    for idx, (data, label) in enumerate(train_loader):
        data = data.to(device=device)
        label = label.to(device=device)
        
        output = model(data)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
       
    print("epoch : %d/%d" % ( epoch, num_epochs))
        
        
        

epoch : 0/10
epoch : 1/10
epoch : 2/10
epoch : 3/10
epoch : 4/10
epoch : 5/10
epoch : 6/10
epoch : 7/10
epoch : 8/10
epoch : 9/10


In [43]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking the train data")
    else:
        print("checking the test data")
        
    num_correct = 0
    num_samples = 0 
    model.eval()
        
    with torch.no_grad():
        for data, label in loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            _, predictions = output.max(1)
            num_correct += (predictions == label).sum()
            num_samples += predictions.size(0)
        print(num_correct/num_samples)
    model.train()
    

In [44]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

checking the train data
tensor(0.9923, device='cuda:0')
checking the test data
tensor(0.9874, device='cuda:0')
